In [1]:
import talos as ta
import gensim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Flatten, Dense, TimeDistributed, \
    SpatialDropout1D, Bidirectional, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, \
    Concatenate, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/data.csv')

In [3]:
df.head()

,word,tag
0,Pengamat,O
1,politik,O
2,dari,O
3,Universitas,B-ORGANIZATION
4,Gadjah,I-ORGANIZATION


In [4]:
sentences = []
cnt = 1

for i in df.itertuples():
    sentences.append(cnt)
    
    if '.' in str(i.word):
        cnt += 1
        
df['sentence #'] = sentences
df.head()

,word,tag,sentence #
0,Pengamat,O,1
1,politik,O,1
2,dari,O,1
3,Universitas,B-ORGANIZATION,1
4,Gadjah,I-ORGANIZATION,1


In [5]:
agg_func = lambda s: [(w, t) for w, t in zip(s['word'].values.tolist(), s['tag'].values.tolist())]
grouped = df.groupby('sentence #').apply(agg_func)
sentences = [s for s in grouped]

In [6]:
words = list(set(df['word'].values))
words.append('PADDING')
num_words = len(words)
tags = list(set(df['tag'].values))
num_tags = len(tags)

In [7]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate([tag for tag in tags if tag != 'O'])}
tag2idx['O'] = len(tags)-1

In [8]:
tag2idx

{'B-QUANTITY': 0,
 'B-TIME': 1,
 'B-LOCATION': 2,
 'I-PERSON': 3,
 'L-LOCATION': 4,
 'L-TIME': 5,
 'L-QUANTITY': 6,
 'B-ORGANIZATION': 7,
 'I-ORGANIZATION': 8,
 'I-TIME': 9,
 'U-LOCATION': 10,
 'U-ORGANIZATION': 11,
 'L-PERSON': 12,
 'U-QUANTITY': 13,
 'U-PERSON': 14,
 'I-LOCATION': 15,
 'U-TIME': 16,
 'B-PERSON': 17,
 'I-QUANTITY': 18,
 'L-ORGANIZATION': 19,
 'O': 20}

In [9]:
max_len = 60
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding='post', value=num_words-1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding='post', value=tag2idx['O'])
y = [to_categorical(i, num_classes=num_tags) for i in y]

In [10]:
x_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=1)

In [11]:
model = gensim.models.Word2Vec.load("../checkpoint/w2vec_wiki_id_case")

In [12]:
embedding_matrix = np.zeros((len(words), 400))

for i, w in enumerate(words):
    try:
        embedding_vector = model.wv[w]
        embedding_matrix[i] = embedding_vector
    except:
        embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), 400)

In [13]:
embedding_matrix.shape

(13031, 400)

In [14]:
params = {
    'optimizer': ['Nadam', 'Adam', 'RMSprop'],
    'lr': [0.01, 0.001, 0.0001],
    'units': [100],
    'filters': [128],
    'dropout': [0.5],
    'batch_size': [64]
}

In [15]:
def create_model(x_train, y_train, x_val, y_val, params):
    input_sequence = Input(shape=(max_len, ))
    model = Embedding(
        input_dim=embedding_matrix.shape[0], 
        weights=[embedding_matrix], 
        output_dim=embedding_matrix.shape[1], 
        input_length=max_len,
        trainable=False
    )(input_sequence)
    model = SpatialDropout1D(params['dropout'])(model)
    model2g = Conv1D(params['filters'], 2, activation='relu', padding='same')(model)
    model4g = Conv1D(params['filters'], 4, activation='relu', padding='same')(model)
    model6g = Conv1D(params['filters'], 6, activation='relu', padding='same')(model)
    model8g = Conv1D(params['filters'], 8, activation='relu', padding='same')(model)
    model10g = Conv1D(params['filters'], 10, activation='relu', padding='same')(model)
    model = concatenate([model2g, model4g, model6g, model8g, model10g])
    model = Bidirectional(
        LSTM(units=params['units'], return_sequences=True, recurrent_dropout=params['dropout'])
    )(model)
    output_sequence = Dense(num_tags, activation='softmax')(model)
    model = Model(input_sequence, output_sequence)
    
    f1_score_m = tfa.metrics.F1Score(
        num_classes=num_tags,
        average='micro',
        name='f1_score',
        threshold=0.5
    )

    if params['optimizer'] == 'Nadam':
        optm = tf.keras.optimizers.Nadam(lr=params['lr'])
        
    if params['optimizer'] == 'Adam':
        optm = tf.keras.optimizers.Adam(lr=params['lr'])
        
    if params['optimizer'] == 'RMSprop':
        optm = tf.keras.optimizers.RMSprop(lr=params['lr'])
        
    loss = tf.keras.losses.CategoricalCrossentropy()

    model.compile(loss=loss, optimizer=optm, metrics=[f1_score_m])
    
    early_stopping = EarlyStopping(patience=10)

    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size=params['batch_size'],
        epochs=100,
        verbose=1,
        callbacks=[early_stopping]
    )
    
    return history, model

In [16]:
t = ta.Scan(x=x_train,
            y=np.array(y_train),
            model=create_model,
            params=params,
            experiment_name='bilstm_cnns_w2v_multiseq_opt',
            val_split=None,
            x_val=x_val,
            y_val=np.array(y_val)
)

  0%|          | 0/9 [00:00<?, ?it/s]

Train on 3424 samples, validate on 734 samples
Epoch 1/100
3424/3424 [==============================] - 24s 7ms/sample - loss: 0.3205 - f1_score: 0.9376 - val_loss: 0.2021 - val_f1_score: 0.9520
Epoch 2/100
3424/3424 [==============================] - 21s 6ms/sample - loss: 0.1773 - f1_score: 0.9549 - val_loss: 0.1468 - val_f1_score: 0.9608
Epoch 3/100
3424/3424 [==============================] - 21s 6ms/sample - loss: 0.1397 - f1_score: 0.9607 - val_loss: 0.1304 - val_f1_score: 0.9630
Epoch 4/100
3424/3424 [==============================] - 20s 6ms/sample - loss: 0.1134 - f1_score: 0.9664 - val_loss: 0.1035 - val_f1_score: 0.9696
Epoch 5/100
3424/3424 [==============================] - 20s 6ms/sample - loss: 0.0986 - f1_score: 0.9699 - val_loss: 0.1148 - val_f1_score: 0.9643
Epoch 6/100
3424/3424 [==============================] - 22s 6ms/sample - loss: 0.0886 - f1_score: 0.9724 - val_loss: 0.0945 - val_f1_score: 0.9717
Epoch 7/100
3424/3424 [==============================] - 24s 7ms/

 11%|█         | 1/9 [09:04<1:12:32, 544.09s/it]

Train on 3424 samples, validate on 734 samples
Epoch 1/100
3424/3424 [==============================] - 26s 8ms/sample - loss: 0.3723 - f1_score: 0.9351 - val_loss: 0.2558 - val_f1_score: 0.9465
Epoch 2/100
3424/3424 [==============================] - 23s 7ms/sample - loss: 0.2329 - f1_score: 0.9493 - val_loss: 0.2007 - val_f1_score: 0.9531
Epoch 3/100
3424/3424 [==============================] - 23s 7ms/sample - loss: 0.1938 - f1_score: 0.9540 - val_loss: 0.1714 - val_f1_score: 0.9564
Epoch 4/100
3424/3424 [==============================] - 23s 7ms/sample - loss: 0.1661 - f1_score: 0.9580 - val_loss: 0.1470 - val_f1_score: 0.9614
Epoch 5/100
3424/3424 [==============================] - 23s 7ms/sample - loss: 0.1448 - f1_score: 0.9615 - val_loss: 0.1340 - val_f1_score: 0.9641
Epoch 6/100
3424/3424 [==============================] - 25s 7ms/sample - loss: 0.1349 - f1_score: 0.9635 - val_loss: 0.1273 - val_f1_score: 0.9657
Epoch 7/100
3424/3424 [==============================] - 23s 7ms/

 22%|██▏       | 2/9 [22:30<1:12:39, 622.81s/it]

...

Train on 3424 samples, validate on 734 samples
Epoch 1/100
3424/3424 [==============================] - 41s 12ms/sample - loss: 0.3360 - f1_score: 0.9345 - val_loss: 0.2320 - val_f1_score: 0.9485
Epoch 2/100
3424/3424 [==============================] - 32s 9ms/sample - loss: 0.2170 - f1_score: 0.9485 - val_loss: 0.2044 - val_f1_score: 0.9495
Epoch 3/100
3424/3424 [==============================] - 34s 10ms/sample - loss: 0.1746 - f1_score: 0.9536 - val_loss: 0.1474 - val_f1_score: 0.9595
Epoch 4/100
3424/3424 [==============================] - 46s 13ms/sample - loss: 0.1431 - f1_score: 0.9595 - val_loss: 0.1337 - val_f1_score: 0.9628
Epoch 5/100
3424/3424 [==============================] - 37s 11ms/sample - loss: 0.1236 - f1_score: 0.9633 - val_loss: 0.1121 - val_f1_score: 0.9682
Epoch 6/100
3424/3424 [==============================] - 38s 11ms/sample - loss: 0.1072 - f1_score: 0.9677 - val_loss: 0.1099 - val_f1_score: 0.9680
Epoch 7/100
3424/3424 [==============================] - 37s

 67%|██████▋   | 6/9 [1:23:29<44:46, 895.34s/it]

Train on 3424 samples, validate on 734 samples
Epoch 1/100
3424/3424 [==============================] - 34s 10ms/sample - loss: 0.6778 - f1_score: 0.8289 - val_loss: 0.3581 - val_f1_score: 0.9300
Epoch 2/100
3424/3424 [==============================] - 30s 9ms/sample - loss: 0.3384 - f1_score: 0.9366 - val_loss: 0.3082 - val_f1_score: 0.9432
Epoch 3/100
3424/3424 [==============================] - 30s 9ms/sample - loss: 0.3053 - f1_score: 0.9426 - val_loss: 0.2873 - val_f1_score: 0.9452
Epoch 4/100
3424/3424 [==============================] - 28s 8ms/sample - loss: 0.2888 - f1_score: 0.9442 - val_loss: 0.2753 - val_f1_score: 0.9457
Epoch 5/100
3424/3424 [==============================] - 27s 8ms/sample - loss: 0.2778 - f1_score: 0.9449 - val_loss: 0.2660 - val_f1_score: 0.9468
Epoch 6/100
3424/3424 [==============================] - 29s 9ms/sample - loss: 0.2689 - f1_score: 0.9456 - val_loss: 0.2572 - val_f1_score: 0.9474
Epoch 7/100
3424/3424 [==============================] - 29s 8ms

Epoch 56/100
3424/3424 [==============================] - 34s 10ms/sample - loss: 0.0998 - f1_score: 0.9701 - val_loss: 0.1046 - val_f1_score: 0.9694
Epoch 57/100
3424/3424 [==============================] - 29s 8ms/sample - loss: 0.0992 - f1_score: 0.9706 - val_loss: 0.1042 - val_f1_score: 0.9695
Epoch 58/100
3424/3424 [==============================] - 29s 8ms/sample - loss: 0.0970 - f1_score: 0.9708 - val_loss: 0.1036 - val_f1_score: 0.9695
Epoch 59/100
3424/3424 [==============================] - 29s 8ms/sample - loss: 0.0973 - f1_score: 0.9707 - val_loss: 0.1030 - val_f1_score: 0.9697
Epoch 60/100
3424/3424 [==============================] - 29s 8ms/sample - loss: 0.0943 - f1_score: 0.9715 - val_loss: 0.1019 - val_f1_score: 0.9701
Epoch 61/100
3424/3424 [==============================] - 31s 9ms/sample - loss: 0.0935 - f1_score: 0.9716 - val_loss: 0.1002 - val_f1_score: 0.9702
Epoch 62/100
3424/3424 [==============================] - 29s 9ms/sample - loss: 0.0919 - f1_score: 0.972

KeyboardInterrupt: 

In [ ]:
t.data[['loss', 'f1_score', 'val_loss', 'val_f1_score', 'optimizer', 'lr']] \
    .sort_values(by=['val_f1_score'], ascending=False)

In [40]:
from sklearn.metrics import f1_score

In [41]:
preds = []
actuals = []

for i, xt in enumerate(x_test):
    p = model.predict(np.array([xt]))
    p = np.argmax(p, axis=-1)
    y_true = np.argmax(np.array(y_test), axis=-1)[i]
    
    for true, pred in zip(y_true, p[0]):
        actuals.append(true)
        preds.append(pred)
        
print(f1_score(preds, actuals, average='micro'))
print(precision_score(preds, actuals, average='micro'))
print(recall_score(preds, actuals, average='micro'))

0.974591280653951
0.974591280653951
0.974591280653951


In [43]:
preds = []
actuals = []

for i, xt in enumerate(x_test):
    p = model.predict(np.array([xt]))
    p = np.argmax(p, axis=-1)
    y_true = np.argmax(np.array(y_test), axis=-1)[i]
    
    if 13030 in list(xt):
        first_pad_idx = list(xt).index(13030)
        _zip = zip(y_true[:first_pad_idx], p[0][:first_pad_idx])
    else:
        _zip = zip(y_true, p[0])
    
    for true, pred in _zip:
        actuals.append(true)
        preds.append(pred)

print(f1_score(preds, actuals, average='micro'))
print(precision_score(preds, actuals, average='micro'))
print(recall_score(preds, actuals, average='micro'))

0.9225551941310818
0.9225551941310818
0.9225551941310818
